In [1]:
import json
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

tqdm.pandas()



In [2]:
with open("data_source/data.json", 'r', encoding='utf-8') as f:
    clean_data = json.load(f)

print(len(clean_data))
print(clean_data[0].keys())
print(json.dumps(clean_data[0], indent=2, ensure_ascii=False))

9996
dict_keys(['source', 'text', 'date'])
{
  "source": "fontanka.ru",
  "text": "Купели для крещенского омовения будут организованы также в Красногвардейском и Приморском районах. Предварительный список адресов 17 января публикует ГУ МЧС по Петербургу.Опубликованный на сайте Смольного список дополнили купелями у церкви Покрова Пресвятой Богородицы на проспекте Косыгина и у яхт-клуба «Геркулес» на Беговой улице, 19. Время проведения омовений по этим адресам не уточняется.Ранее представители Русской православной церкви призвали верующих отказаться от погружения в прорубь во время Крещения Господня в разгар коронавируса. Они напомнили, что омовение не является церковным ритуалом. Официальный представитель Иваново-Вознесенской епархии Макарий (Маркиш) и вовсе назвал обычай маргинальным.Несколько российских регионов ввели запрет на купания в этом году из-за распространения коронавируса. В Петербурге, находящемся на втором месте по количеству заболевших в стране, такое решение не принято. 

In [4]:
df = pd.DataFrame(clean_data)
df['date'] = pd.to_datetime(df['date'])

print(f"Loaded {len(df):,} articles from {df['source'].nunique()} sources")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(df['source'].value_counts())

Loaded 9,996 articles from 4 sources
Date range: 2020-01-01 00:00:00 to 2021-12-31 00:00:00
source
kommersant.ru    3528
lenta.ru         3528
fontanka.ru      1764
ria.ru           1176
Name: count, dtype: int64


In [8]:
df.head(10)

,source,text,date
0,fontanka.ru,Купели для крещенского омовения будут организо...,2021-01-17
1,fontanka.ru,Следственный комитет Ленинградской области под...,2021-01-10
2,fontanka.ru,Индекс Московской биржи обновил исторический м...,2021-01-11
3,fontanka.ru,В комитете по благоустройству призвали петербу...,2021-01-14
4,fontanka.ru,В России зафиксирован первый случай заражения ...,2021-01-10
5,fontanka.ru,Уполномоченный при президенте России по правам...,2021-01-31
6,fontanka.ru,"Ночной пожар, затронувший две легковые машины,...",2021-01-06
7,fontanka.ru,Следователи разбираются в обстоятельствах преж...,2021-01-09
8,fontanka.ru,Комитет по здравоохранению прокомментировал жа...,2021-01-11
9,fontanka.ru,Фотокорреспондент «Фонтанки» прогулялся по цен...,2021-01-04


In [12]:
df['text_length'] = df['text'].str.len()
print(df['text_length'].describe())

count     9996.000000
mean      2777.082133
std       3121.393725
min         13.000000
25%       1023.750000
50%       1524.500000
75%       4055.000000
max      60678.000000
Name: text_length, dtype: float64


## Sparse методы

### Расстояние Левенштейна

In [14]:
from Levenshtein import distance as lev_distance 

In [15]:
def normalized_levenshtein(s1, s2):
    max_len = max(len(s1), len(s2))
    if max_len == 0:
        return 0
    return lev_distance(s1, s2) / max_len 

In [19]:
sample_indices = df.sample(5, random_state=42).index
print("Sample Levenshtein distances:")
for i in range(len(sample_indices)-1):
    idx1, idx2 = sample_indices[i], sample_indices[i+1]
    text1, text2 = df.loc[idx1, 'text'], df.loc[idx2, 'text']
    dist = normalized_levenshtein(text1, text2)
    print(f"Pair {i+1}: {dist:.4f}")
    print(f"  Text 1 ({len(text1)} chars): {text1[:500]}...")
    print(f"  Text 2 ({len(text2)} chars): {text2[:500]}...\n")

Sample Levenshtein distances:
Pair 1: 0.7986
  Text 1 (1104 chars): Суперконтейнеровоз Ever Given, заблокировавший в марте движение по Суэцкому каналу, возвращается из Роттердама и вновь попытается пройти тем же маршрутом. «Контейнеровоз готовится пересечь водный путь с севера на юг. Судно возвращается из Роттердама, куда оно прибыло в конце июля после урегулирования всех финансовых вопросов с каналом и достижения мирового соглашения. В Порт-Саид контейнеровоз должен зайти сегодня вечером, проход начнется рано утром, — отметили источники агентства. — Будут приня...
  Text 2 (3049 chars): В интересах Ваших клиентов Генеральному директору ООО «Т2 Мобайл» С. В. Эмдину Уважаемый Сергей Владимирович! 10 июня Ваша компания планирует сменить провайдера. Русфонд получил письмо от ООО «ТЕКО» с сообщением о выборе Вашим предприятием этой компании в качестве провайдера для работы с благотворительными фондами. ООО «ТЕКО» предлагает заключить соответствующий договор о партнерстве. Подобные письма п

In [21]:
from collections import defaultdict

levenshtein_pairs = []
df['month'] = df['date'].dt.to_period('M')

for month in tqdm(df['month'].unique(), desc="Processing months"):
    mask = (df['month'] >= month - 1) & (df['month'] <= month + 1)
    indices = df[mask].index.tolist()
    
    if len(indices) < 2:
        continue
    
    texts = {idx: df.loc[idx, 'text'] for idx in indices}
    lengths = {idx: len(texts[idx]) for idx in indices}
    
    for i in range(len(indices)):
        for j in range(i+1, len(indices)):
            idx1, idx2 = indices[i], indices[j]
            

            len_ratio = min(lengths[idx1], lengths[idx2]) / max(lengths[idx1], lengths[idx2])
            if len_ratio < 0.5:
                continue
            
            text1_start = texts[idx1][:100].lower()
            text2_start = texts[idx2][:100].lower()
            quick_dist = lev_distance(text1_start, text2_start) / 100
            if quick_dist > 0.7:
                continue
            
            text1, text2 = texts[idx1], texts[idx2]
            lev_dist = normalized_levenshtein(text1, text2)
            lev_sim = 1 - lev_dist
            
            if lev_sim > 0.5: 
                levenshtein_pairs.append((idx1, idx2, lev_sim))

print(f"\nComputed Levenshtein for {len(levenshtein_pairs):,} pairs")


Processing months:   0%|          | 0/24 [00:00<?, ?it/s]


Computed Levenshtein for 2,103 pairs


In [22]:
levenshtein_pairs_sorted = sorted(levenshtein_pairs, key=lambda x: x[2], reverse=True)

for idx, (i, j, sim) in enumerate(levenshtein_pairs_sorted[:10], 1):
    print(f"\n{idx}. Similarity: {sim:.4f}")
    print(f"\nArticle {i} [{df.loc[i, 'source']}, {df.loc[i, 'date'].date()}]:")
    print(df.loc[i, 'text'])
    print(f"\nArticle {j} [{df.loc[j, 'source']}, {df.loc[j, 'date'].date()}]:")
    print(df.loc[j, 'text'])
    print("\n" + "="*80)


1. Similarity: 0.9980

Article 2258 [kommersant.ru, 2020-04-01]:
Официальные курсы ЦБ России на 01.04.20 Австралийский доллар 47,1448 Английский фунт 94,5771 Белорусский рубль 30,1979 Датская крона* 11,4760 Доллар США 77,7325 Евро 85,7389 Индийская рупия** 10,3457 Казахский тенге** 17,4278 Канадский доллар 55,2941 Китайский юань* 10,9611 Норвежская крона* 73,7766 СДР 105,7869 Сингапурский доллар 54,2673 Новая турецкая лира 12,0661 Украинская гривна* 27,5213 Шведская крона* 78,0000 Швейцарский франк 80,7191 Японская иена** 71,4027 *За 10. **За 100.

Article 2269 [kommersant.ru, 2020-04-02]:
Официальные курсы ЦБ России на 02.04.20 Австралийский доллар 47,1448 Английский фунт 94,5771 Белорусский рубль 30,1979 Датская крона* 11,4760 Доллар США 77,7325 Евро 85,7389 Индийская рупия** 10,3457 Казахский тенге** 17,4278 Канадский доллар 55,2941 Китайский юань* 10,9611 Норвежская крона* 73,7766 СДР 105,7869 Сингапурский доллар 54,2673 Новая турецкая лира 12,0661 Украинская гривна* 27,5213 Шведс

### Лемматизация + TF-IDF/BM25

In [24]:
import pymorphy3 

morph = pymorphy3.MorphAnalyzer()

In [25]:
def lemmatize_text(text):
    tokens = text.lower().split()
    lemmas = [morph.parse(token)[0].normal_form for token in tokens]
    return ' '.join(lemmas)

In [27]:
sample_text = df.iloc[0]['text'][:650]
print(sample_text)
lemmatized_text = lemmatize_text(sample_text)
print(f"\nЛемматизация: {lemmatized_text}")

Купели для крещенского омовения будут организованы также в Красногвардейском и Приморском районах. Предварительный список адресов 17 января публикует ГУ МЧС по Петербургу.Опубликованный на сайте Смольного список дополнили купелями у церкви Покрова Пресвятой Богородицы на проспекте Косыгина и у яхт-клуба «Геркулес» на Беговой улице, 19. Время проведения омовений по этим адресам не уточняется.Ранее представители Русской православной церкви призвали верующих отказаться от погружения в прорубь во время Крещения Господня в разгар коронавируса. Они напомнили, что омовение не является церковным ритуалом. Официальный представитель Иваново-Вознесенско

Лемматизация: купель для крещенский омовение быть организовать также в красногвардейский и приморский районах. предварительный список адрес 17 январь публиковать гу мчс по петербургу.опубликовать на сайт смольный список дополнить купель у церковь покров пресвятой богородица на проспект косыгин и у яхт-клуб «геркулес» на бегов улице, 19. время про

In [28]:
df['text_lemmatized'] = df['text'].progress_apply(lemmatize_text)

  0%|          | 0/9996 [00:00<?, ?it/s]

Лемматизировали весь датасет


In [29]:
df[['text_lemmatized']].to_csv('data_source/lemmatized_texts.csv', index=True)

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_features=5000,
    min_df=2,
    max_df=0.8,
    ngram_range=(1, 2),
    lowercase=True,
    strip_accents='unicode',
    stop_words=None
)

tfidf_lemma = vectorizer.fit_transform(df['text_lemmatized'])
print(tfidf_lemma.shape)
print(len(vectorizer.get_feature_names_out()))

(9996, 5000)
5000


In [31]:
dense_matrix_lemma = tfidf_lemma.toarray()
print(dense_matrix_lemma)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [33]:
non_zero_mask_lemma = (dense_matrix_lemma != 0).any(axis=1)
dense_matrix_lemma_filtered = dense_matrix_lemma[non_zero_mask_lemma]
indices_lemma_filtered = np.where(non_zero_mask_lemma)[0]

print(f"Нулевые векторы:{(~non_zero_mask_lemma).sum()}")
print(f"Ненулевые векторы:{(non_zero_mask_lemma).sum()}")

Нулевые векторы:0
Ненулевые векторы:9996


In [34]:
from sklearn.cluster import AgglomerativeClustering

clustering_lemma = AgglomerativeClustering(
    n_clusters=None,
    distance_threshold=0.4,
    linkage='average',
    metric='cosine'
)

clusters_lemma_filtered = clustering_lemma.fit_predict(dense_matrix_lemma_filtered)


In [35]:
clusters_tfidf_lemma = np.full(len(dense_matrix_lemma), -1)
clusters_tfidf_lemma[indices_lemma_filtered] = clusters_lemma_filtered

df['cluster_tfidf_lemma'] = clusters_tfidf_lemma

In [36]:
print(f"\nЧисло кластеров: {len(np.unique(clusters_tfidf_lemma))}")
print(f"Распределение кластеров по размеру:")
print(pd.Series(clusters_tfidf_lemma).value_counts().head(20))


Число кластеров: 9434
Распределение кластеров по размеру:
877    76
4      24
46     23
141    13
29     12
208    10
62      9
14      8
184     8
49      8
108     8
234     7
79      7
982     6
87      6
130     5
92      5
438     5
396     5
144     5
Name: count, dtype: int64


In [37]:
duplicate_clusters_tfidf = df['cluster_tfidf_lemma'].value_counts()
duplicate_clusters_tfidf = duplicate_clusters_tfidf[duplicate_clusters_tfidf > 1]

print(f"Всего кластеров: {len(np.unique(clusters_tfidf_lemma))}")
print(f"Кластеры с дубликатами: {len(duplicate_clusters_tfidf)}")
print(f"Общая сумма статей-дубликатов: {duplicate_clusters_tfidf.sum()}")
print(f"\n10 самых больших кластеров:")
print(duplicate_clusters_tfidf.head(10))

Всего кластеров: 9434
Кластеры с дубликатами: 279
Общая сумма статей-дубликатов: 841

10 самых больших кластеров:
cluster_tfidf_lemma
877    76
4      24
46     23
141    13
29     12
208    10
62      9
14      8
184     8
49      8
Name: count, dtype: int64


In [43]:
valid_clusters_tfidf = duplicate_clusters_tfidf[duplicate_clusters_tfidf.index != -1]

if len(valid_clusters_tfidf) > 0:
    largest_cluster = valid_clusters_tfidf.idxmax()
    cluster_items = df[df['cluster_tfidf_lemma'] == largest_cluster].copy()
    
    print(f"\nСамый большой кластер: {largest_cluster}; Число статей: {len(cluster_items)}\n")
    for idx, (i, row) in enumerate(cluster_items.head(20).iterrows(), 1):
        print(f"\nСтатья {idx} [{row['source']}, {row['date'].date()}]:")
        print(row['text'][:500] + "..." if len(row['text']) > 500 else row['text'])
        print("="*80)



Самый большой кластер: 877; Число статей: 76


Статья 1 [kommersant.ru, 2020-01-16]:
Официальные курсы ЦБ России на 16.01.20 Австралийский доллар 42,2965 Английский фунт 79,9548 Белорусский рубль 28,9177 Датская крона* 91,4451 Доллар США 61,4328 Евро 68,3747 Индийская рупия** 86,7449 Казахский тенге** 16,2060 Канадский доллар 47,0137 Китайский юань* 89,2024 Норвежская крона* 69,1507 СДР 84,8154 Сингапурский доллар 45,5834 Новая турецкая лира 10,4155 Украинская гривна* 25,6510 Шведская крона* 64,8443 Швейцарский франк 63,5293 Японская иена** 55,8836 *За 10. **За 100.

Статья 2 [kommersant.ru, 2020-01-28]:
Официальные курсы ЦБ России на 28.01.20 Австралийский доллар 42.3337 Английский фунт 81.5194 Белорусский рубль 29.4228 Датская крона* 92.0200 Доллар США 62.3380 Евро 68.7775 Индийская рупия** 87.2684 Казахский тенге** 16.3821 Канадский доллар 47.3226 Китайский юань* 89.8669 Норвежская крона* 68.7049 СДР 85.8425 Сингапурский доллар 46.0263 Новая турецкая лира 10.4879 Украинская гривна*

Теперь BM25

In [ ]:
#from rank_bm25 import BM25Okapi

corpus_lemmatized = [text.split() for text in df['text_lemmatized']]

In [53]:
import bm25s

retriever = bm25s.BM25()
retriever.index(corpus_lemmatized)


resource module not available on Windows


BM25S Create Vocab:   0%|          | 0/9996 [00:00<?, ?it/s]

BM25S Convert tokens to indices:   0%|          | 0/9996 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/9996 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/9996 [00:00<?, ?it/s]

In [ ]:
from collections import defaultdict

month_to_indices = defaultdict(set)
for idx, row in df.iterrows():
    month_to_indices[row['month']].add(idx)

def get_window_indices(month):
    indices = set()
    for m in [month - 1, month, month + 1]:
        indices.update(month_to_indices.get(m, set()))
    return indices

bm25_pairs = []

for idx in tqdm(range(len(corpus_lemmatized)), desc="BM25 pairs"):
    query = corpus_lemmatized[idx]
    scores = retriever.get_scores(query)  
    
    month = df.loc[idx, 'month']
    window_indices = get_window_indices(month)
    
    for idx2 in window_indices:
        if idx2 > idx and scores[idx2] > 30.0:
            bm25_pairs.append((idx, idx2, scores[idx2]))

print(f"\nTotal pairs: {len(bm25_pairs):,}")

BM25 pairs:   0%|          | 0/9996 [00:00<?, ?it/s]


Total pairs: 2,579,420


In [57]:
if len(bm25_pairs) > 0:
    bm25_scores = [score for _, _, score in bm25_pairs]
    
    print(f"BM25 score statistics:")
    print(f"  Mean: {np.mean(bm25_scores):.2f}")
    print(f"  Median: {np.median(bm25_scores):.2f}")
    print(f"  Std: {np.std(bm25_scores):.2f}")
    print(f"  Min: {np.min(bm25_scores):.2f}")
    print(f"  Max: {np.max(bm25_scores):.2f}")
    
    print(f"\nPairs by score threshold:")
    for threshold in [5, 10, 15, 20, 30, 50, 100]:
        count = sum(1 for score in bm25_scores if score > threshold)
        print(f"  > {threshold}: {count:,} pairs")


BM25 score statistics:
  Mean: 55.30
  Median: 40.48
  Std: 51.78
  Min: 20.00
  Max: 3851.36

Pairs by score threshold:
  > 5: 2,579,420 pairs
  > 10: 2,579,420 pairs
  > 15: 2,579,420 pairs
  > 20: 2,579,420 pairs
  > 30: 1,818,164 pairs
  > 50: 963,090 pairs
  > 100: 246,185 pairs


In [64]:
from unionfind import unionfind

bm25_threshold = 300.0
filtered_bm25_pairs = [(i, j, s) for i, j, s in bm25_pairs if s > bm25_threshold]
print(f"Число пар с порогом > {bm25_threshold}: {len(filtered_bm25_pairs):,}")

uf_bm25 = unionfind(len(df))

for idx1, idx2, score in filtered_bm25_pairs:
    uf_bm25.unite(idx1, idx2)

index_to_cluster_bm25 = {}
clusters_bm25 = []

for idx in range(len(df)):
    root = uf_bm25.find(idx) 
    if root not in index_to_cluster_bm25:
        index_to_cluster_bm25[root] = len(index_to_cluster_bm25)
    clusters_bm25.append(index_to_cluster_bm25[root])

df['cluster_bm25'] = clusters_bm25
print(f"Число кластеров: {len(np.unique(clusters_bm25))}")

Число пар с порогом > 300.0: 18,585
Число кластеров: 4518


In [65]:

duplicate_clusters_bm25 = df['cluster_bm25'].value_counts()
duplicate_clusters_bm25 = duplicate_clusters_bm25[duplicate_clusters_bm25 > 1]

print(f"Всего кластеров: {len(np.unique(clusters_bm25))}")
print(f"Кластеры с дубликатами: {len(duplicate_clusters_bm25)}")
print(f"Всего статей-дубликатов: {duplicate_clusters_bm25.sum()}")
if len(duplicate_clusters_bm25) > 0:
    print(f"Средний размер кластера с дубликатами: {duplicate_clusters_bm25.mean():.2f}")
    print(f"Самый большой кластер: {duplicate_clusters_bm25.max()}")
    print(f"\n10 самых больших кластеров:")
    print(duplicate_clusters_bm25.head(10))


Всего кластеров: 4518
Кластеры с дубликатами: 17
Всего статей-дубликатов: 5495
Средний размер кластера с дубликатами: 323.24
Самый большой кластер: 5455

10 самых больших кластеров:
cluster_bm25
34      5455
4180       8
226        3
1609       3
2117       2
1588       2
1579       2
1671       2
1608       2
1966       2
Name: count, dtype: int64


In [66]:

print("="*80)
print("Кластеры с дубликатами")
print("="*80)

for cluster_id in duplicate_clusters_bm25.index:
    cluster_items = df[df['cluster_bm25'] == cluster_id].copy()
    
    print(f"\n{'='*80}")
    print(f"Кластер {cluster_id}; Число статей: {len(cluster_items)}")
    print(f"{'='*80}")
    
    for idx, (i, row) in enumerate(cluster_items.head(15).iterrows(), 1):
        print(f"\n{idx}. Статья {i} [{row['source']}, {row['date'].date()}]:")
        print(row['text'][:500] + "..." if len(row['text']) > 500 else row['text'])
        print("-"*80)


Кластеры с дубликатами

Кластер 34; Число статей: 5455

1. Статья 34 [fontanka.ru, 2021-01-22]:
Елизавета Юхнёва побывала внутри двух с лишним сотен картин художников разных стран и воссоздала полотна своими руками. А ещё она работала на горячей линии Красного Креста и трудится волонтером в больнице для бездомных.Одной из примет коронавирусного времени стала игра в повторение сюжетов известных картин. Сидя дома на карантине, люди в разных странах стали выкладывать в соцсети фотоколлажи, одну половину которых занимает изображение какого-нибудь шедевра живописи, а другую — собственная попытк...
--------------------------------------------------------------------------------

2. Статья 40 [fontanka.ru, 2021-01-07]:
Пандемия COVID-19 не оставит камня на камне от привычного нам мира. Уже вскоре нас ждет медицинский фашизм, уход в виртуальность, попадание в рабство к искусственному интеллекту и железный занавес 2.0.Общемировой локдаун и вызванный им экономический кризис — результат ошибки в 

### Выводы

Для полных или практически полных дубликатов sparse подходы работают достаточно хорошо, но в общем случае при их использовании новости не-дубликаты сваливаются в один кластер. В следующих разделах посмотрим на dense методы. 